In [ ]:
import pandas as pd
import numpy as np
from pandas.api.types import is_datetime64_any_dtype

df = pd.read_csv('.//customers-1000.csv')#df is the data frame containing the whole content of file
columns = np.array(df.columns)#to know the number of colums in dataframe(csv) file
noOfColumns = len(columns)
noOfRows = len(df.index)
print(f"Total columns = {noOfColumns}")
print(f"Total rows = {noOfRows}")
sortAccordingToDateTime = True
sortAccordingToComment = False

if sortAccordingToDateTime == True:
    df = clusterDateTimeCol(df) 
    

print(df)

In [ ]:
    def clusterDateTimeCol(fContent):
        for col in fContent.columns:
            a = sort_if_year_column(fContent[col])
            if a == False:
                pass
            else:
                fContent[col] = a
            
            try:
                # Try to convert the column to datetime (auto format detection)
                #it is flexible for all type of date time format
                parsed_col = pd.to_datetime(fContent[col], errors='raise', dayfirst=True)
                # Check if all time values are 00:00:00 → meaning only dates
                if all(parsed_col.dt.time == pd.to_datetime('00:00:00').time()): #it is for date only 
                    print("date")
                    fContent = clean_and_sort_date_column(df,col)
                else:# it is for both date and time
                    print(f"{col} dateTime")
                    fContent = Shandle_datetime_column(df,col)
            except Exception:
                continue
    
        return fContent 

In [12]:
#if Only year
def sort_if_year_column(column):
    try:
        # Step 1: Convert to string and strip whitespace
        cleaned = column.astype(str).str.strip()
        
        # Step 2: Check if all values match 4-digit pattern
        if cleaned.str.fullmatch(r'\d{4}').all():
            # Step 3: Convert to int and check valid year range
            years = cleaned.astype(int)
            if years.between(1800, 2050).all():
                print("✅ Year-only column detected. Sorted.")
                return years.sort_values(ignore_index=True)
            else:
               pass
        else:
            pass
    except Exception as e:
        print(f"Error: {e}")
    
    return False # Return as-is if not valid


In [13]:
#fuction which will sort the dates if the df has date containing columns
def clean_and_sort_date_column(dff, column_name, ascending=True):
    try:
         # Step 1: Convert to datetime (handles multiple formats)
        dff[column_name] = pd.to_datetime(dff[column_name], errors='coerce', dayfirst=True)
        
        # Step 2: Drop NaT (invalid formats)
        dff = dff.dropna(subset=[column_name])
        
        # Step 3: Sort the DataFrame by that column
        dff = dff.sort_values(by=column_name, ascending=ascending)

        # Optional: Format to clean date string (YYYY-MM-DD)
        # dff[column_name] = dff[column_name].dt.strftime('%Y-%m-%d')

        return dff
    
    except Exception as e:
        print(f"⚠️ Error while processing date column: {e}")
        return dff

In [14]:
# if df has column containing the date and time both
def handle_datetime_column(df, column_name):

    # Check if most values in column are datetime with time
    values = df[column_name].dropna().astype(str).head(20)
    count_datetime = sum([is_datetime(v) for v in values])

    if count_datetime >= len(values) // 2:  # At least half must be datetime-like
        # Convert full column to datetime
        df[column_name] = pd.to_datetime(df[column_name], errors='coerce')
        # Drop rows with invalid dates
        df = df.dropna(subset=[column_name])
        # Sort by that column
        df = df.sort_values(by=column_name).reset_index(drop=True)
        print(f"[INFO] '{column_name}' successfully recognized and sorted as datetime.")
    else:
        print(f"[INFO] '{column_name}' does not contain proper datetime with time.")

    return df

In [ ]:
#Group(MultiIndexing) accordind to the condition 